In [ ]:
import torch
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

# إعداد الأنابيب للنموذج - Setting up the pipeline for the model


In [ ]:
pipe = pipeline(
    "text-generation",
    model="MBZUAI-Paris/Atlas-Chat-2B",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=-1
)

# إعداد مجموعة من الرسائل لاختبار النموذج - Preparing a set of messages to test the model

In [ ]:
messages = [
    {"role": "user", "content": "ما هو الذكاء الاصطناعي؟"},
    {"role": "user", "content": "كيف يمكنني تحسين مهارات البرمجة؟"},
    {"role": "user", "content": "ما هي مميزات النموذج الذي تستخدمه؟"},
    {"role": "user", "content": "اكتب قصة قصيرة عن مغامرة."},
    {"role": "user", "content": "اشرح لي نظرية فيزيائية معقدة."}
]

# إعداد قائمة لتخزين النتائج - Preparing a list to store the results

In [ ]:
results = []

# تجربة النموذج مع إعدادات مختلفة - Testing the model with different settings

In [ ]:
temperatures = [0.0, 0.3, 0.7, 1.0]  # قيم درجات الحرارة المختلفة - Different temperature values
max_new_tokens_list = [50, 100, 200]  # حدود مختلفة لطول النص - Different text length limits

# قياس الأداء والزمن - Measuring performance and time

In [ ]:
for temp in temperatures:
    for max_tokens in max_new_tokens_list:
        for message in messages:
            start_time = time.time()  # وقت البداية - Start time
            
            # استخراج النص من الرسالة - Extracting text from the message
            message_text = message["content"]
            
            # تمرير النص إلى النموذج - Passing text to the model
            output = pipe(
                message_text,  # استخدم النص فقط Use the text only
                max_new_tokens=max_tokens,
                temperature=temp
            )
            response_time = time.time() - start_time  # زمن الاستجابة - Response time

            # تخزين النتائج - Storing results
            results.append({
                "message": message_text,  # تخزين النص فقط - Store the text only
                "response": output[0]["generated_text"].strip(),
                "temperature": temp,
                "max_tokens": max_tokens,
                "response_time": response_time
            })


# تحويل النتائج إلى إطار بيانات لتحليلها - Converting results to a DataFrame for analysis

In [ ]:
results_df = pd.DataFrame(results)

# حفظ النتائج كملف CSV - Saving results as a CSV file

In [ ]:
results_df.to_csv("atlas_results.csv", index=False)

# إنشاء الرسوم البيانية - Creating visualizations

In [ ]:
sns.set(style="whitegrid")

# رسم العلاقة بين الحرارة والزمن - Plotting the relationship between temperature and response time

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="temperature", y="response_time", data=results_df)
plt.title("Effect of Temperature on Response Time")
plt.xlabel("Temperature")
plt.ylabel("Response Time (seconds)")
plt.savefig("temperature_vs_time.png")
plt.show()

# رسم العلاقة بين طول النص والزمن - Plotting the relationship between max tokens and response time

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="max_tokens", y="response_time", data=results_df)
plt.title("Effect of Max Tokens on Response Time")
plt.xlabel("Max Tokens")
plt.ylabel("Response Time (seconds)")
plt.savefig("tokens_vs_time.png")
plt.show()

# رسم توزيع طول النصوص الناتجة - Plotting the distribution of generated text lengths

In [ ]:
results_df["response_length"] = results_df["response"].apply(len)  # حساب طول النصوص - Calculate text lengths
plt.figure(figsize=(10, 6))
sns.histplot(results_df, x="response_length", hue="temperature", kde=True, bins=20)
plt.title("Distribution of Generated Text Lengths by Temperature")
plt.xlabel("Response Length (characters)")
plt.ylabel("Frequency")
plt.savefig("response_length_distribution.png")
plt.show()

# عرض أول 5 نتائج - Displaying the first 5 results

In [ ]:
print(results_df.head())